In [20]:
import cupy as cp
from cupyx import scatter_add
import cupyx.scipy.sparse.linalg
import cupyx.scipy.sparse as cusparse
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from includes.preprocessor import write_keywords,write_birth,write_parameters
from includes.gamma import domain_mgr, heat_solve_mgr,load_toolpath,get_toolpath
%matplotlib notebook
cp.cuda.Device(1).use()
!nvidia-smi
import pyvista as pv
from pyvirtualdisplay import Display
import vtk

Thu May 27 13:39:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.56       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:01:00.0 Off |                  Off |
| 34%   41C    P8    27W / 260W |    195MiB / 48598MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     On   | 00000000:21:00.0 Off |                  Off |
| 33%   

In [21]:
domain = domain_mgr(filename='input_files/thinwall.k',toolpath_file='input_files/thinwall_toolpath.crs')

Time of reading input files: 4.6107823848724365
Time of calculating critical timestep: 0.012678146362304688
Time of reading and interpolating toolpath: 0.019801616668701172
Number of nodes: 40444
Number of elements: 34192
Number of time-steps: 25065
Time of generating surface: 26.253082036972046


In [38]:
def elastic_stiff_matrix(elements, nodes, shear_0, bulk_0):
    n_n = nodes.shape[0]
    n_e = elements.shape[0]
    n_p = elements.shape[1]
    n_q = 8
    n_int = n_e*n_q
    nodes_pos = nodes[elements]
    Jac = cp.matmul(domain.Bip_ele,nodes_pos[:,cp.newaxis,:,:].repeat(8,axis=1)) # J = B*x [B:8(nGP)*3(dim)*8(nN), x:nE*8*8*3]
    ele_detJac = cp.linalg.det(Jac)
    iJac = cp.linalg.inv(Jac) #inv J (nE*nGp*dim*dim)
    ele_gradN = cp.matmul(iJac,domain.Bip_ele) # dN/dx = inv(J)*B

    ele_B = cp.zeros([n_e,n_q,6,n_p*3])
    ele_B[:,:,0,0:24:3] = ele_gradN[:,:,0,:]
    ele_B[:,:,1,1:24:3] = ele_gradN[:,:,1,:]
    ele_B[:,:,2,2:24:3] = ele_gradN[:,:,2,:]
    ele_B[:,:,3,0:24:3] = ele_gradN[:,:,1,:]
    ele_B[:,:,3,1:24:3] = ele_gradN[:,:,0,:]
    ele_B[:,:,4,1:24:3] = ele_gradN[:,:,2,:]
    ele_B[:,:,4,2:24:3] = ele_gradN[:,:,1,:]
    ele_B[:,:,5,2:24:3] = ele_gradN[:,:,0,:]
    ele_B[:,:,5,0:24:3] = ele_gradN[:,:,2,:]

    temp = cp.array([[0,1,2]]).repeat(n_p,axis=0).flatten()
    jB = 3*cp.tile(elements[:,cp.newaxis,cp.newaxis,:],(1,n_q,6,1)).repeat(3,axis=3) + temp
    vB = ele_B.reshape(-1,n_p*3)
    jB = jB.reshape(-1,n_p*3)
    iB = cp.arange(0,jB.shape[0])[:,cp.newaxis].repeat(n_p*3,axis=1)
    B = cusparse.csr_matrix((cp.ndarray.flatten(vB),(cp.ndarray.flatten(iB), cp.ndarray.flatten(jB))), shape = (6*n_int, 3*n_n), dtype = cp.float)

    IOTA = cp.array([[1],[1],[1],[0],[0],[0]]) 
    VOL = cp.matmul(IOTA,IOTA.transpose()) 
    DEV = cp.diag([1,1,1,1/2,1/2,1/2])-VOL/3

    ELASTC = 2*DEV*shear_0 + VOL*bulk_0
    ele_D = ele_detJac[:,:,cp.newaxis,cp.newaxis]*ELASTC
    temp = cp.arange(0,n_e*n_q*6).reshape(n_e,n_q,6)
    iD = temp[:,:,cp.newaxis,:].repeat(6,axis = 2)
    jD = temp[:,:,:,cp.newaxis].repeat(6,axis = 3)

    D = cusparse.csr_matrix((cp.ndarray.flatten(ele_D),(cp.ndarray.flatten(iD), cp.ndarray.flatten(jD))), shape = (6*n_int, 6*n_int), dtype = cp.float)
    ele_K =  ele_B.transpose([0,1,3,2])@ele_D@ele_B
    ele_K = ele_K.sum(axis = 1)

#     temp = cp.array([[0,1,2]]).repeat(n_p,axis=0).flatten()[:,cp.newaxis]
#     iK = 3*elements[:,:,cp.newaxis].repeat(3,axis=1) + temp
#     iK = iK.repeat(3*n_p,axis=2)
#     jK = 3*elements[:,cp.newaxis,:].repeat(3,axis=2) + temp.transpose()
#     jK = jK.repeat(3*n_p,axis=1)
#     K  = cusparse.csr_matrix((cp.ndarray.flatten(ele_K),(cp.ndarray.flatten(iK),cp.ndarray.flatten(jK))), shape = (3*n_n, 3*n_n), dtype = cp.float)
    K = B.transpose()*D*B 
    return K,B,D,ele_B,ele_D,iD,jD,ele_detJac

def constitutive_problem(E, Ep_prev, Hard_prev, shear_0, bulk_0, a_0, Y):
    IOTA = cp.array([[1],[1],[1],[0],[0],[0]])  
    VOL = cp.matmul(IOTA,IOTA.transpose()) 
    DEV = cp.diag([1,1,1,1/2,1/2,1/2])-VOL/3
    E_tr = E-Ep_prev  
    ELASTC = 2*DEV*shear_0 + VOL*bulk_0
    S_tr = (ELASTC @ E_tr[:,:,:,cp.newaxis]).squeeze()
    SD_tr = (2*DEV*shear_0@E_tr[:,:,:,cp.newaxis]).squeeze() - Hard_prev
    norm_SD = cp.sqrt(cp.sum(SD_tr[:,:,0:3]*SD_tr[:,:,0:3], axis=2)+2*cp.sum(SD_tr[:,:,3:6]*SD_tr[:,:,3:6], axis=2))

    CRIT = norm_SD-Y
    IND_p = CRIT>0 

    S = cp.array(S_tr)
    DS = cp.ones((S.shape[0],S.shape[1],6,6))*ELASTC

    if not IND_p[IND_p].shape[0]:
        Ep = cp.array(Ep_prev)
        Hard = cp.array(Hard_prev)
        return S, DS, IND_p, Ep, Hard   
        
    N_hat = SD_tr[IND_p]/norm_SD[IND_p][:,cp.newaxis].repeat(6,axis=1)  
    denom =  2*shear_0 + a_0 
    Lambda = CRIT[IND_p]/denom

    S[IND_p] = S[IND_p] - 2*N_hat*shear_0*Lambda[:,cp.newaxis].repeat(6,axis=1)  
    NN_hat = N_hat[:,:,cp.newaxis]@N_hat[:,cp.newaxis,:]
    const = 4*shear_0**2/denom

    DS[IND_p] = DS[IND_p] - const*DEV + (const*Y[IND_p]/norm_SD[IND_p])[:,cp.newaxis,cp.newaxis].repeat(6,axis=1).repeat(6,axis=2)*(DEV-NN_hat)


    Ep = cp.array(Ep_prev)
    Ep[IND_p] = Ep[IND_p]+cp.matmul(cp.array([[1],[1],[1],[2],[2],[2]]),Lambda[cp.newaxis]).transpose()*N_hat

    Hard = cp.array(Hard_prev)
    Hard[IND_p] = Hard[IND_p]+(a_0*Lambda[:,cp.newaxis].repeat(6,axis=1))*N_hat
    
    return S, DS, IND_p, Ep, Hard

In [39]:
nodes = domain.nodes
elements = domain.elements
n_n = len(domain.nodes)
n_e = len(domain.elements)
n_p = 8
n_q = 8
n_int = n_e * n_q 
HatP = domain.Nip_ele.transpose()     # HatP   - values of basis functions at the quadrature points,
                                                          # size(HatP)=(n_p,n_q)
DHatP1 = domain.Bip_ele[:,0,:].transpose()        # DHatP1 - derivatives of basis functions at the quadrature points 
                                                          #  in the direction xi_1, size(DHatP1)=(n_p,n_q)
DHatP2 = domain.Bip_ele[:,1,:].transpose()       # DHatP2 - derivatives of basis functions at the quadrature points 
                                                          #  in the direction xi_2, size(DHatP2)=(n_p,n_q)
DHatP3 = domain.Bip_ele[:,2,:].transpose()         # DHatP3 - derivatives of basis functions at the quadrature points 
                                                          #  in the direction xi_3, size(DHatP3)=(n_p,n_q)
                                                          # n_p    - number of basis functions
                                                          # n_q    - number of integration points within one elemen

# values of elastic material parameters
#young = 206900                        # Young's modulus in MPa
young = 116888                         # Young's modulus at average temperature
poisson =  0.32                        # Poisson's ratio
shear_0 = young/(2*(1+poisson))        # shear modulus
bulk_0 = young/(3*(1-2*poisson))       # bulk modulus
# plastic material parematers
#a_0 = 10000
a_0 = 9500                             # Kinematic hardening parameter
#Y_0 = 450*np.sqrt(2/3)                # Yield stress in MPa
Y_0 = 744                              # Note that this is actually yield stress * sqrt(2/3)
T_Ref = 400                            # Reference or Ambient Temparature, always equal to 300 if specified otherwise
#T_Ref = 252                           # Reference or Ambient Temparature, always equal to 300 if specified otherwise
# Tolerence for Newton stopping criterion
tol = 1.0e-8                           # non-dimensionalized tolerence 
# Maximum Number of N_R Iterations allowed
Maxit = 150

# Initialization for the whole boundary-value problem
E = cp.zeros((n_e,n_q,6))                        # strain tensors at integration points
Ep_prev = cp.zeros((n_e,n_q,6))                  # plastic strain tensors at integration points
Hard_prev = cp.zeros((n_e,n_q,6))
Y = Y_0 * cp.ones((n_e,n_q))
U = cp.zeros((n_n,3))
dU = cp.zeros((n_n,3))
F = cp.zeros((n_n,3))
f = cp.zeros((n_n,3))                          
scl = 1.14875854e-05                           # average thermal expansion coefficient for Ti64A              
alpha_Th = cp.array([scl,scl,scl,0,0,0])
idirich = cp.array(nodes[:, 2] == -20.0 ) 

In [40]:
%%time
heat_solver = heat_solve_mgr(domain)
domain.current_time = 0
endtime = 150
timestep = int(endtime/domain.dt)+1
# timestep = 1
heat_solver.q_in = 200
file_num = 0

layer_time = [11,23,35,47,59,71,83,95,107,119]
layer = 0
n_e_active = sum(domain.element_birth < layer_time[layer])
n_n_active = sum(domain.node_birth < layer_time[layer])
active_eles = elements[0:n_e_active]
active_nodes = nodes[0:n_n_active]
Q = cp.zeros(nodes.shape, dtype=bool)
Q[0:n_n_active,:] = 1 
Q[idirich,:] = 0 
K_elast,B,D_elast,ele_B,ele_D,iD,jD,ele_detJac = elastic_stiff_matrix(active_eles,active_nodes, shear_0, bulk_0)

for t in range(0,timestep):
    if t % 5000 == 0:
        mempool = cp.get_default_memory_pool()
        mempool.free_all_blocks()
        print("Current time:  {}, Percentage done:  {}%".format(domain.current_time,100*t/timestep))  
    heat_solver.time_integration()
    
    if t % 20 == 0:
        if domain.current_time>layer_time[layer]:
            layer = layer + 1
            U = disp_match(nodes,U,layer)
            n_e_active = sum(domain.element_birth < layer_time[layer])
            n_n_active = sum(domain.node_birth < layer_time[layer])
            active_eles = elements[0:n_e_active]
            active_nodes = nodes[0:n_n_active]
            K_elast,B,D_elast,ele_B,ele_D,iD,jD,ele_detJac = elastic_stiff_matrix(active_eles,active_nodes, shear_0, bulk_0)

        temperature_nodes = cp.clip(heat_solver.temperature[elements[0:n_e_active]],300,2300) - 300
        temperature_ip = (domain.Nip_ele[:,cp.newaxis,:]@temperature_nodes[:,cp.newaxis,:,cp.newaxis].repeat(8,axis=1))[:,:,0,0]
        it = 0
        U_it = U[0:n_n_active,:]
        while True:
            #E[0:n_e_active] = (ele_B@(U[active_eles].reshape(-1,24))[:,cp.newaxis,:,cp.newaxis].repeat(n_q,axis=1)).squeeze()
            E[0:n_e_active] = cp.reshape(B@U_it.flatten(),(-1,8,6))
            E[0:n_e_active] = E[0:n_e_active] - temperature_ip[:,:,cp.newaxis].repeat(6,axis=2)*alpha_Th
            
            S, DS, IND_p,_,_ = constitutive_problem(E[0:n_e_active], Ep_prev[0:n_e_active], Hard_prev[0:n_e_active], shear_0, bulk_0, a_0, Y[0:n_e_active])
            vD = ele_detJac[:,:,cp.newaxis,cp.newaxis].repeat(6,axis=2).repeat(6,axis=3) * DS
            D_p = cusparse.csr_matrix((cp.ndarray.flatten(vD), (cp.ndarray.flatten(iD),cp.ndarray.flatten(jD))), shape = D_elast.shape, dtype = cp.float)
            K_tangent = K_elast + B.transpose()*(D_p-D_elast)*B
            n_plast = len(IND_p[IND_p])
            print(' plastic integration points: ', n_plast, ' of ', IND_p.shape[0]*IND_p.shape[1])
            F = B.transpose() @ ((ele_detJac[:,:,cp.newaxis].repeat(6,axis=2)*S).reshape(-1))
            dU[Q,:],error = cusparse.linalg.cg(K_tangent[Q[0:n_n_active].flatten()][:,Q[0:n_n_active].flatten()],-F[Q[0:n_n_active].flatten()],tol=tol)
#            dU,error = cusparse.linalg.cg(K_tangent,-F,tol=tol)
            U_new = U_it + dU[0:n_n_active,:] 
#            U_new[0:n_n_active] = U_new[0:n_n_active] + dU.reshape(-1,3)
            q1 = dU[0:n_n_active].flatten()@K_elast@dU[0:n_n_active].flatten()
            q2 = U[0:n_n_active].flatten()@K_elast@U[0:n_n_active].flatten()
            q3 = U_new[0:n_n_active].flatten()@K_elast@U_new[0:n_n_active].flatten()
            if q2 == 0 and q3 == 0:
                criterion = 0
            else:
                criterion = q1/(q2+q3)
                print('  stopping criterion=  ', criterion)

            U_it = cp.array(U_new) 
            # test on the stopping criterion
            if  criterion < tol:
                break
                
            # test on number of iteration
            it = it+1
            if  it > Maxit:
                raise Exception('The Newton solver does not converge for the current timestep: {}'.format(t))
        U[0:n_n_active] = U_it        
        E[0:n_e_active] = (ele_B@(U[active_eles].reshape(-1,24))[:,cp.newaxis,:,cp.newaxis].repeat(n_q,axis=1)).squeeze()
        E[0:n_e_active] = E[0:n_e_active] - temperature_ip[:,:,cp.newaxis].repeat(6,axis=2)*alpha_Th
            
        S, DS, IND_p,Ep,Hard = constitutive_problem(E[0:n_e_active], Ep_prev[0:n_e_active], Hard_prev[0:n_e_active], shear_0, bulk_0, a_0, Y[0:n_e_active])

        Ep_prev[0:n_e_active] = Ep
        Hard_prev[0:n_e_active] = Hard
        
        if t%200 == 0:
            filename = 'residual_stress/thinwall/thinwall_{}.vtk'.format(file_num)
            save_vtk(filename)
            file_num = file_num + 1

Current time:  0, Percentage done:  0.0%
 plastic integration points:  0  of  187136
 plastic integration points:  0  of  187136
 plastic integration points:  0  of  187136
 plastic integration points:  0  of  187136
 plastic integration points:  0  of  187136
 plastic integration points:  0  of  187136
 plastic integration points:  0  of  187136
  stopping criterion=   0.9999999999999999
 plastic integration points:  54  of  187136
  stopping criterion=   0.00019059960075066336
 plastic integration points:  58  of  187136
  stopping criterion=   2.9592126405488853e-07
 plastic integration points:  58  of  187136
  stopping criterion=   2.3357638180692296e-13
 plastic integration points:  21  of  187136
  stopping criterion=   0.17555326009317204
 plastic integration points:  95  of  187136
  stopping criterion=   0.00010281096715532723
 plastic integration points:  101  of  187136
  stopping criterion=   2.808713882533387e-07
 plastic integration points:  102  of  187136
  stopping cr

 plastic integration points:  258  of  187136
  stopping criterion=   5.263338073012019e-07
 plastic integration points:  260  of  187136
  stopping criterion=   1.3576918761213039e-11
 plastic integration points:  121  of  187136
  stopping criterion=   0.04103835069437412
 plastic integration points:  264  of  187136
  stopping criterion=   0.00013216263016670255
 plastic integration points:  276  of  187136
  stopping criterion=   4.956710863450154e-07
 plastic integration points:  276  of  187136
  stopping criterion=   4.5627890256988715e-12
 plastic integration points:  113  of  187136
  stopping criterion=   0.03767866997255621
 plastic integration points:  262  of  187136
  stopping criterion=   0.00014175338584049272
 plastic integration points:  261  of  187136
  stopping criterion=   1.2785692246285583e-06
 plastic integration points:  262  of  187136
  stopping criterion=   1.243887127848225e-10
 plastic integration points:  103  of  187136
  stopping criterion=   0.0411054

  stopping criterion=   4.4677571443301575e-11
 plastic integration points:  146  of  187136
  stopping criterion=   0.033078502011380725
 plastic integration points:  357  of  187136
  stopping criterion=   0.00013295462137510256
 plastic integration points:  351  of  187136
  stopping criterion=   1.0870148349372262e-06
 plastic integration points:  351  of  187136
  stopping criterion=   3.3000256879906444e-11
 plastic integration points:  138  of  187136
  stopping criterion=   0.03732775545525846
 plastic integration points:  372  of  187136
  stopping criterion=   0.00012856289419935533
 plastic integration points:  387  of  187136
  stopping criterion=   9.113886834592942e-07
 plastic integration points:  384  of  187136
  stopping criterion=   1.8051925860176344e-11
 plastic integration points:  149  of  187136
  stopping criterion=   0.030575894526167595
 plastic integration points:  380  of  187136
  stopping criterion=   0.00011607947176604632
 plastic integration points:  3

 plastic integration points:  506  of  187136
  stopping criterion=   0.00010720827681328446
 plastic integration points:  510  of  187136
  stopping criterion=   3.280892555377047e-07
 plastic integration points:  512  of  187136
  stopping criterion=   7.55374607319039e-11
 plastic integration points:  200  of  187136
  stopping criterion=   0.029000935001443313
 plastic integration points:  508  of  187136
  stopping criterion=   0.00011588386286714299
 plastic integration points:  516  of  187136
  stopping criterion=   1.0700946927695885e-06
 plastic integration points:  516  of  187136
  stopping criterion=   1.7002038686021464e-10
 plastic integration points:  225  of  187136
  stopping criterion=   0.031054666614217654
 plastic integration points:  511  of  187136
  stopping criterion=   0.00013609593715321113
 plastic integration points:  523  of  187136
  stopping criterion=   3.5237847699652505e-06
 plastic integration points:  532  of  187136
  stopping criterion=   3.33704

  stopping criterion=   9.758914333021098e-07
 plastic integration points:  631  of  187136
  stopping criterion=   3.299358540805132e-11
 plastic integration points:  227  of  187136
  stopping criterion=   0.030957915828328672
 plastic integration points:  640  of  187136
  stopping criterion=   0.00011757357137314297
 plastic integration points:  657  of  187136
  stopping criterion=   8.401215740403889e-07
 plastic integration points:  653  of  187136
  stopping criterion=   8.991044402490523e-12
 plastic integration points:  245  of  187136
  stopping criterion=   0.02587478174864722
 plastic integration points:  645  of  187136
  stopping criterion=   0.00010643476810084778
 plastic integration points:  657  of  187136
  stopping criterion=   1.380680589081256e-06
 plastic integration points:  658  of  187136
  stopping criterion=   3.2958458509112187e-10
 plastic integration points:  255  of  187136
  stopping criterion=   0.02737091078892985
 plastic integration points:  651  o

 plastic integration points:  500  of  196736
  stopping criterion=   0.008107162471575982
 plastic integration points:  1089  of  196736
  stopping criterion=   7.20710971738385e-05
 plastic integration points:  1097  of  196736
  stopping criterion=   1.7008859292956425e-07
 plastic integration points:  1097  of  196736
  stopping criterion=   1.0243998108536511e-12
 plastic integration points:  500  of  196736
  stopping criterion=   0.002981497544102782
 plastic integration points:  1158  of  196736
  stopping criterion=   4.42153012984541e-05
 plastic integration points:  1163  of  196736
  stopping criterion=   5.823527833646127e-08
 plastic integration points:  1163  of  196736
  stopping criterion=   7.29557630642098e-14
 plastic integration points:  529  of  196736
  stopping criterion=   0.0014734872623740872
 plastic integration points:  1239  of  196736
  stopping criterion=   2.9338972982092386e-05
 plastic integration points:  1224  of  196736
  stopping criterion=   1.23

  stopping criterion=   0.00965874663012335
 plastic integration points:  1864  of  196736
  stopping criterion=   0.00031393727097469274
 plastic integration points:  1893  of  196736
  stopping criterion=   1.8038490015900162e-06
 plastic integration points:  1891  of  196736
  stopping criterion=   6.911428419873885e-11
 plastic integration points:  854  of  196736
  stopping criterion=   0.008251981925405443
 plastic integration points:  1895  of  196736
  stopping criterion=   0.0003025469090662951
 plastic integration points:  1936  of  196736
  stopping criterion=   1.441250030025172e-06
 plastic integration points:  1942  of  196736
  stopping criterion=   1.8129187606592518e-10
 plastic integration points:  883  of  196736
  stopping criterion=   0.008676019421870695
 plastic integration points:  1964  of  196736
  stopping criterion=   0.0003491705703830259
 plastic integration points:  2013  of  196736
  stopping criterion=   2.4395529949850124e-06
 plastic integration point

In [5]:
def save_vtk(filename):
    active_elements = domain.elements[domain.active_elements].tolist()
    active_cells = np.array([item for sublist in active_elements for item in [8] + sublist])
    active_cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(active_elements))
    points = domain.nodes[0:n_n_active].get() + 5*U[0:n_n_active].get()
    Sv =  transformation(cp.sqrt(1/2*((S[:,:,0]-S[:,:,1])**2 + (S[:,:,1]-S[:,:,2])**2 + (S[:,:,2]-S[:,:,0])**2 + 6*(S[:,:,3]**2+S[:,:,4]**2+S[:,:,5]**2))),active_eles, ele_detJac)
    S11 = transformation(S[:,:,0], active_eles, ele_detJac)
    S22 = transformation(S[:,:,1], active_eles, ele_detJac)
    S33 = transformation(S[:,:,2], active_eles, ele_detJac)
    S12 = transformation(S[:,:,3], active_eles, ele_detJac)
    S23 = transformation(S[:,:,4], active_eles, ele_detJac)
    S13 = transformation(S[:,:,5], active_eles, ele_detJac)
    active_grid = pv.UnstructuredGrid(active_cells, active_cell_type, points)
    active_grid.point_arrays['temp'] = heat_solver.temperature[0:n_n_active].get()
    active_grid.point_arrays['S_von'] = Sv.get()
    active_grid.point_arrays['S11'] = S11.get()
    active_grid.point_arrays['S22'] = S22.get()
    active_grid.point_arrays['S33'] = S33.get()
    active_grid.point_arrays['S12'] = S12.get()
    active_grid.point_arrays['S23'] = S23.get()
    active_grid.point_arrays['S13'] = S13.get()
    active_grid.point_arrays['U1'] = U[0:n_n_active,0].get()
    active_grid.point_arrays['U2'] = U[0:n_n_active,1].get()
    active_grid.point_arrays['U3'] = U[0:n_n_active,2].get()
    active_grid.save(filename)

In [6]:
def transformation(Q_int, active_elements, ele_detJac):
    Q_int = Q_int.reshape(1,-1)
    elem = cp.array(active_elements.transpose())                      # elements.transpose() with shape (n_p=8,n_e)
    weight = ele_detJac.reshape(1,-1)
    #n_n = COORD.shape[1]          # number of nodes including midpoints
    n_e = elem.shape[1]            # number of elements
    n_p = 8                        # number of vertices per element
    n_q = 8                        # number of quadrature points
    n_int = n_e*n_q                # total number of integrations points
    # values at integration points, shape(vF1)=shape(vF2)=(n_p,n_int)   
    vF1 = cp.matmul(cp.ones((n_p,1)), weight*Q_int)    
    vF2 = cp.matmul(cp.ones((n_p,1)),weight)

    # row and column indices, shape(iF)=shape(jF)=(n_p,n_int)   
    iF = cp.zeros((n_p,n_int), dtype=cp.int32)         ######
    jF = cp.kron(elem, cp.ones((1,n_q), dtype=cp.int32))

    # the asssembling by using the sparse command - values v for duplicate
    # doubles i,j are automatically added together
    F1 = cusparse.csr_matrix((cp.ndarray.flatten(vF1.transpose()),(cp.ndarray.flatten(iF.transpose()), cp.ndarray.flatten(jF.transpose()))), dtype = cp.float) 
    F2 = cusparse.csr_matrix((cp.ndarray.flatten(vF2.transpose()),(cp.ndarray.flatten(iF.transpose()), cp.ndarray.flatten(jF.transpose()))), dtype = cp.float) 

    #
    # Approximated values of the function Q at nodes of the FE mesh
    #
    Q = cp.array(F1/F2)
    Q_node = cp.ones(Q.shape[1])
    Q_node[0:n_n_active] = Q[0,0:n_n_active]
    return Q_node

In [7]:
def disp_match(nodes,U,layer):
    idar = cp.arange(nodes.shape[0])
    U1 = cp.array(U)
    pre_nodes = domain.node_birth < layer_time[layer-1]
    cur_nodes = (domain.node_birth >= layer_time[layer-1]) * (domain.node_birth < layer_time[layer])
    zel = nodes[cur_nodes,2].max()
    zel_prev = nodes[pre_nodes,2].max()
    for k in idar[cur_nodes]:
        U1[k,:] = U[int(idar[(nodes[:,0] == nodes[k,0]) * (nodes[:,1] == nodes[k,1]) * (nodes[:,2] == zel_prev)]),:]
    return U1